### MSDS  7337-404 - Natural Language Processing ###

### Homework2 ###
**Shah Dhyan**

In [5]:
import sys;
print('This is python:')
print(sys.version)
print('')

import nltk; print( 'nltk ' + nltk.__version__)
from nltk import word_tokenize;

import bs4; print( 'bs4 ' + bs4.__version__)
from bs4 import BeautifulSoup, SoupStrainer;

import re; print('re ' + re.__version__)
import requests; print('requests ' + requests.__version__)

This is python:
3.7.4 (default, Aug 13 2019, 15:17:50) 
[Clang 4.0.1 (tags/RELEASE_401/final)]

nltk 3.4.5
bs4 4.8.0
re 2.2.1
requests 2.22.0


## Answer 1 ## 
**The simple approach considered is to divide each text length by the highest vocabulary size. This helps interpret results with a linear scaling.**

In [46]:
url = 'https://www.gutenberg.org/wiki/Technology_(Bookshelf)'

In [60]:
def get_text_from_url(url):
    return requests.get(url).text

In [48]:
def get_all_links_from_html(html):
    tags = BeautifulSoup(html, 'html.parser', parse_only=SoupStrainer('a', href=True))
    urls = [str(tag.attrs['href']) for tag in tags]
    return urls

In [49]:
def relevent_link(link):
    if '/ebooks/' in link:
        return True
    return False

In [61]:
def get_book_ids_from_links(links):
    return [link.split('/ebooks/').pop() for link in links]

In [51]:
def get_book_text_urls_from_ids(ids):
    url_template = 'http://www.gutenberg.org/files/{}/{}.txt'
    # url_template = 'http://www.gutenberg.org/files/{}/{}-h/{}-h.htm'
    return [url_template.format(id, id, id) for id in ids]

In [86]:
all_texts = []

def fetch_all_texts():
    if len(all_texts) > 0:
        print(len(all_texts))
        return
    # fetch the html from the home page 
    home_page_html = get_text_from_url(url_to_fetch_images)
    # collect links from the html
    all_links_from_home_page = get_all_links_from_html(home_page_html)
    # removing links that can't be used
    relevent_links_from_home_page = list(filter(relevent_link, all_links_from_home_page))

    book_ids = get_book_ids_from_links(relevent_links_from_home_page)
    book_urls = get_book_text_urls_from_ids(book_ids)
    print('found {} books'.format(len(book_urls)))
    book_i = 0
    for url in book_urls:
        print ('fetching book via {}'.format(url), end=' ')
        book_text = get_text_from_url(url)
        print('found text with {} characters'.format(len(book_text)))
        all_texts.append({
            'url': url,
            'corpus': book_text
        })

        book_i += 1
        # maximum number of book fetch
        if book_i == 10:
            return
        
    
    # adding text to dictionary 
    return 1
fetch_all_texts()

found 233 books
fetching book via http://www.gutenberg.org/files/19533/19533.txt found text with 115097 characters
fetching book via http://www.gutenberg.org/files/24681/24681.txt found text with 125855 characters
fetching book via http://www.gutenberg.org/files/33912/33912.txt found text with 89433 characters
fetching book via http://www.gutenberg.org/files/18448/18448.txt found text with 4430 characters
fetching book via http://www.gutenberg.org/files/21281/21281.txt found text with 58979 characters
fetching book via http://www.gutenberg.org/files/23319/23319.txt found text with 427253 characters
fetching book via http://www.gutenberg.org/files/29269/29269.txt found text with 63846 characters
fetching book via http://www.gutenberg.org/files/27632/27632.txt found text with 392837 characters
fetching book via http://www.gutenberg.org/files/25269/25269.txt found text with 283990 characters
fetching book via http://www.gutenberg.org/files/32282/32282.txt found text with 107219 characters

In [87]:
def get_words_from_corpus(corpus):
    # useing regex to only consider alphabets 
    corpus = re.sub('[^a-zA-Z\s]', '', corpus)
    # select lowercase of all letter
    corpus = corpus.lower()
    # tokanizing words
    return word_tokenize(corpus)

In [88]:
def get_unique_words(words):
    return set(words)

In [90]:
get_unique_words('shah')

{'a', 'h', 's'}

In [91]:
def get_vocabulary_score(vocab_size, highest_vocab_size):
    return float(vocab_size) / highest_vocab_size

get_vocabulary_score(10, 100)

0.1

## Answer 2 ## 
**To obtain the long word score in each book corpus, simply counted the long words per corpus and divided it by the highest of the score.**



In [92]:
def get_long_word_count(vocab):
    minimum_size = 10
    long_words = [t for t in vocab if len(t) >= minimum_size]
    return len(long_words)

get_long_word_count(['tc', 'MeteorologicalInstruments'])

1

In [93]:
def get_lexical_diversity(words, unique_words):
    return float(len(unique_words)) / len(words)

## Answer 3 ## 
**To calculated number of unique words and divided by total # of words for lexical diversity. Then calcualed the length of unique words to derive the vocabulary size and highest number of vocabulary to calculate the vocabulary score. Similarly, calculated # of unique words and maximum number of word counts to derive long word score. Following to that added Lexical Diversity, Vocab. Score and Long Word Score to calculate the difficilty of text.**

In [94]:
def score_vocabulary_size():
    highest_vocab = 0
    highest_long_word_count = 0
    
    # calculate vocabulary size 
    # calculate highest vocabulary size 
    for text in all_texts:
        corpus = text['corpus']
        words = get_words_from_corpus(corpus)
        unique_words = get_unique_words(words)

        vocabulary_size = len(unique_words)
        long_word_count = get_long_word_count(unique_words)
        lexical_diversity = get_lexical_diversity(words, unique_words)

        text['vocabulary_size'] = vocabulary_size
        text['long_word_count'] = long_word_count
        text['lexical_diversity'] = lexical_diversity

        if vocabulary_size > highest_vocab:
            highest_vocab = vocabulary_size
        if long_word_count > highest_long_word_count:
            highest_long_word_count = long_word_count

    # score each item
    for text in all_texts:
        vocabulary_size = text['vocabulary_size']
        long_word_count = text['long_word_count']
        lexical_diversity = text['lexical_diversity']

        vocabulary_score = get_vocabulary_score(vocabulary_size, highest_vocab)
        long_word_score = get_vocabulary_score(long_word_count, highest_long_word_count)
        text_difficulty = vocabulary_score + long_word_score + lexical_diversity

        text['vocabulary_score'] = vocabulary_score
        text['long_word_score'] = long_word_score
        print('text_diff: {0:.1f}, v_score: {1:.1f}, lw_score: {2:.1f}, lex_diver: {3:.1f}, url: {4}'.format(text_difficulty, vocabulary_score, long_word_score, lexical_diversity, text['url']))

score_vocabulary_size()

text_diff: 1.0, v_score: 0.5, lw_score: 0.3, lex_diver: 0.1, url: http://www.gutenberg.org/files/19533/19533.txt
text_diff: 1.5, v_score: 0.7, lw_score: 0.7, lex_diver: 0.2, url: http://www.gutenberg.org/files/24681/24681.txt
text_diff: 1.1, v_score: 0.4, lw_score: 0.4, lex_diver: 0.2, url: http://www.gutenberg.org/files/33912/33912.txt
text_diff: 0.6, v_score: 0.0, lw_score: 0.1, lex_diver: 0.5, url: http://www.gutenberg.org/files/18448/18448.txt
text_diff: 1.0, v_score: 0.4, lw_score: 0.4, lex_diver: 0.2, url: http://www.gutenberg.org/files/21281/21281.txt
text_diff: 2.1, v_score: 1.0, lw_score: 1.0, lex_diver: 0.1, url: http://www.gutenberg.org/files/23319/23319.txt
text_diff: 1.0, v_score: 0.4, lw_score: 0.3, lex_diver: 0.2, url: http://www.gutenberg.org/files/29269/29269.txt
text_diff: 1.4, v_score: 0.6, lw_score: 0.7, lex_diver: 0.1, url: http://www.gutenberg.org/files/27632/27632.txt
text_diff: 1.3, v_score: 0.7, lw_score: 0.5, lex_diver: 0.1, url: http://www.gutenberg.org/files

## Summary ## 

**Used technology book shelf and it appears that**

## Mechanical Drawing Self-Taught , by Joshua Rose ##

**has highest text difficulty.**

**while comparing to Homework 1 grade books LD, it appears that the technology/engineering book appears to have higher lexical diversity which theoratically make sense**

**Example LD from Homework 1**

*text_diversity 0.0013633158348350701 | level:first*

*text_diversity 0.0005173951828724353 | level:third*

*text_diversity 0.00014538990101921624 | level:fifth*

